**Project Codes**

In [1]:
!pip install newspaper3k
!pip install spacy==2.1.0
!pip install neuralcoref
!python -m spacy download en_core_web_lg #you will need to install this on first load
!pip install cdqa

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
! git clone https://github.com/sumehta/question-generation.git

Cloning into 'question-generation'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 91 (delta 16), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (91/91), done.


In [3]:
cd /content/question-generation

/content/question-generation


In [0]:
!unzip QuestionGeneration.zip

In [23]:
cd /content/

/content


In [24]:
!git clone https://github.com/kushalj001/FNC

Cloning into 'FNC'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 29 (delta 4), reused 24 (delta 1), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [2]:
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_lg')
neuralcoref.add_to_pipe(nlp)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
import requests
import re
import numpy as np
import pandas as pd
import time
import math
from bs4 import BeautifulSoup
import newspaper

# import nltk
# #nltk.download("all")
# from nltk.tokenize import sent_tokenize

from ast import literal_eval
from cdqa.utils.download import download_model
from cdqa.pipeline.cdqa_sklearn import QAPipeline

import tensorflow as tf
import tensorflow_hub as hub

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

# Download models
download_model(model='bert-squad_1.1', dir='./models')

# Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
cdqa_pipeline = QAPipeline(reader='models/bert_qa_vCPU-sklearn.joblib')


from question_generation.question_generator import QuestionGenerator

qdf=pd.read_csv("/content/questiontable.csv",encoding='latin')



bert_qa_vCPU-sklearn.joblib already downloaded


In [0]:
doc="""An Israeli company based in the northern coastal city of Ceasarea has developed a revolutionary minimal technique to freeze cancerous tumors that only requires a local anesthetic.

It is already treating thousands of women with breast cancer worldwide within hospitals and doctors’ offices, including in Japan, Ausralia, Germany and Spain. A clinical trial is currently being performed in the United States.

Vice President of Icecure Medical Elizabeth Sadka explains that the method is meant “to replace the surgery for a specific population that we can offer to treat in a minimal invasive operation.”

A solution at a temperature of -274 degrees Fahrenheit is injected into the cancerous tumor to freeze it, rendering it inactive and enabling the body to dispense with it as it does with other dead cells naturally within a few weeks.

Only a minimal scar remains in the tissue while no other trace is left on the remainder of the body or organs.

Most patients can be treated within one hour and in some cases even within just ten to fifteen minutes, allowing them to go home the same day or the following day."""

In [0]:
doc

'An Israeli company based in the northern coastal city of Ceasarea has developed a revolutionary minimal technique to freeze cancerous tumors that only requires a local anesthetic.\n\nIt is already treating thousands of women with breast cancer worldwide within hospitals and doctors’ offices, including in Japan, Ausralia, Germany and Spain. A clinical trial is currently being performed in the United States.\n\nVice President of Icecure Medical Elizabeth Sadka explains that the method is meant “to replace the surgery for a specific population that we can offer to treat in a minimal invasive operation.”\n\nA solution at a temperature of -274 degrees Fahrenheit is injected into the cancerous tumor to freeze it, rendering it inactive and enabling the body to dispense with it as it does with other dead cells naturally within a few weeks.\n\nOnly a minimal scar remains in the tissue while no other trace is left on the remainder of the body or organs.\n\nMost patients can be treated within on

In [0]:
 for para in doc.split("\n"):
        if(para):
        
          print("*",para)

* An Israeli company based in the northern coastal city of Ceasarea has developed a revolutionary minimal technique to freeze cancerous tumors that only requires a local anesthetic.
* It is already treating thousands of women with breast cancer worldwide within hospitals and doctors’ offices, including in Japan, Ausralia, Germany and Spain. A clinical trial is currently being performed in the United States.
* Vice President of Icecure Medical Elizabeth Sadka explains that the method is meant “to replace the surgery for a specific population that we can offer to treat in a minimal invasive operation.”
* A solution at a temperature of -274 degrees Fahrenheit is injected into the cancerous tumor to freeze it, rendering it inactive and enabling the body to dispense with it as it does with other dead cells naturally within a few weeks.
* Only a minimal scar remains in the tissue while no other trace is left on the remainder of the body or organs.
* Most patients can be treated within one ho

In [0]:
def google_search(query):
    try: 
        from googlesearch import search 
    except ImportError:  
        print("No module named 'google' found") 
 
    List=[]  
    for j in search(query, tld="co.in", num=10, stop=10, pause=2): 
        List.append(j) 
    return List

  
def scrapped_dcos(links):

    documents=[]
    flag=0

    URL=[]
    for i in links:
        article = newspaper.Article(i, language='en')
        try:

            article.download()
            article.parse()
            documents.append(article.text)
            URL.append([i])

        except:
            flag=1
            
    return(documents,URL)
  
  
def getresolvedparas(documents,URL):

    pattern='\[.*?\]'
    pattern1='\(.*?\)'

    paragraphs=[]
    linklist=[]
    

    for url,doc in zip(URL,documents):
      #print("docs",doc)
      for para in doc.split("\n"):
        #print("paras",para)
        if(para): 

          para = re.sub(pattern, '', para)
          para = re.sub(pattern1, '', para)
          para = para.replace('\t', ' ').replace('\xa0',' ').replace(".",". ").replace(" , , ",",").replace(", ,",",")
          para = ' '.join(para.split())

          doc = nlp(para)	
          

          #paragraphs.append([doc._.coref_resolved])
          
          paragraphs.append([para])
          
          linklist.extend(url)
          
    
    
    return(paragraphs,linklist)


In [35]:
factcheck("millennials are not buying cars any more and this change in thier mindset is the reason for automobile crisis")

('millennials are not buying cars any more and this change in thier mindset is the reason for automobile crisis', '“Millennials might be buying cars but sharing them more often, or they may have cars they use on weekends only and therefore invest less in it, because they’re going to use ride-share during the week,” Jillson said. “The place the vehicle is in the mindset of younger consumers is shifting because they are being offered more opportunities and more options. ”', 'https://www.mcall.com/business/mc-biz-cars-millennials-20181112-story.html', 'Agree')
('millennials are not buying cars any more and this change in thier mindset is the reason for automobile crisis', '“Millennials might be buying cars but sharing them more often, or they may have cars they use on weekends only and therefore invest less in it, because they’re going to use ride-share during the week,” said Jillson. “The place the vehicle is in the mindset of younger consumers is shifting because they are being offered 

In [0]:
doc=nlp("modi is the prime minister of India. He was also the chiefminister of Gujarat")
print([doc._.coref_resolved])

['modi is the prime minister of India. modi was also the chiefminister of Gujarat']


In [0]:

def factcheck(claim):
  
    answers=[]

    links=google_search(str(claim))
    documents,URL=scrapped_dcos(links)
    paragraphs,linklist=getresolvedparas(documents,URL)

    import pandas as pd
    df= pd.DataFrame(columns=['title','paragraphs'])

    df['paragraphs']=paragraphs
    df['title']=linklist
    

    # Fitting the retriever to the list of documents in the dataframe
    cdqa_pipeline.fit_retriever(df)
    
    #question_list1,question_list2=qans_generator(claim)
    
    #print("printing the list of questions",question_list1,question_list2)
    
    answers.append(get_results(claim))
    
    
#     for i in question_list1:
      
#         answers.append(get_results(claim,i["Q"],i["A"]))
        
    
#     for i in question_list2:
      
      
#         answers.append(get_results(claim,i[0],i[1]))
    
   
    for i in answers:
      
      for j in i:
        
        print(j)
            
    

In [0]:
def elmo_vectors(x):
  
    embeddings = elmo(x,signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        return sess.run(embeddings)

def simscore(str1,str2,ind1,ind2):
  
  v1=elmo_vectors([str1])
  v2=elmo_vectors([str2])
  
  return(cosine_similarity(v1[0,ind1,:].reshape(1,1024), v2[0,ind2,:].reshape(1,1024)).flatten()[0])

In [0]:
def generatequestion(text):
  
    tokenizedsent=[]

    qalist=[]

    
    doc = nlp(text)
    
    for token in doc:
        tokenizedsent.append(token.text)
        
    for ent in doc.ents:
      
         #print(ent,ent.label_)
        
        if(ent.label_=="NORP"): 

            ind1=tokenizedsent.index(ent.text)
            sentence= ' '.join(token for token in tokenizedsent)


            s1=simscore(sentence,"congress is a political group",ind1,3)
            s2=simscore(sentence,"i am a global citizen",ind1,4)
            s3=simscore(sentence,"christianity is a religion",ind1,3)

            if((s1>s2) & (s1>s3)):

                qalist.append((ent.text,text.replace(ent.text,"which political party")))

            elif((s2>s1) & (s2>s3)):

                qalist.append((ent.text,text.replace(ent.text,"which country")))

            else:

                qalist.append((ent.text,text.replace(ent.text,"which religion")))

        else:                   

            qalist.append((text.replace(ent.text,list(qdf.loc[qdf.type==ent.label_,"question"])[0]),ent.text))
            
    
    return(qalist)

In [0]:
def qans_generator(claim):
  

  q  = QuestionGenerator()
  
  question_list1 = q.generate_question(claim)
  
  question_list2 = generatequestion(claim)
  
  
  return(question_list1,question_list2)

  

In [0]:
def get_results(claim):
  
  
  #prediction=cdqa_pipeline.predict(query=question, n_predictions=3)
  
  prediction=cdqa_pipeline.predict(query=claim, n_predictions=3)
  
  #answer=[]
  stance=[]
  paragraphs=[]

  for i in range(0,3):

    #answer.append(format(prediction[i][0]))  #answer
    website =format(prediction[i][1])  #website
    para = format(prediction[i][2])  #paragraph
    
    paragraphs.append(para)
    
    stance.append(getstance(claim,para,website))
        
    
#   for i in range(0,3):
    
#     print("\n")
    
#     print("question ",claim)
#     print("answer ",format(prediction[i][0]))  #answer
#     print("website ",format(prediction[i][1]))  #website
#     print("paragraph ",format(prediction[i][2]))  #paragraph
 
  return(stance)
    

In [0]:
def get_results1(claim,question,actualanswer):
  
  
  #prediction=cdqa_pipeline.predict(query=question, n_predictions=3)
  
  prediction=cdqa_pipeline.predict(query=claim, n_predictions=3)
  
  #answer=[]
  stance=[]
  paragraphs=[]

  for i in range(0,3):

    #answer.append(format(prediction[i][0]))  #answer
    website =format(prediction[i][1])  #website
    para = format(prediction[i][2])  #paragraph
    
    paragraphs.append(para)
    
    stance.append(getstance(claim,para,website))
        
    
#   for i in range(0,3):
    
#     print("\n")
    
#     print("question ",claim)
#     print("answer ",format(prediction[i][0]))  #answer
#     print("website ",format(prediction[i][1]))  #website
#     print("paragraph ",format(prediction[i][2]))  #paragraph
 
  return(claim,paragraphs,stance)
    


In [0]:
getstance("rajni has affair with silk smitha","Strongest among the rumours from Rajini's heydays in films is that he was romantically involved with Silk Smitha","www.rumours.com")

printing paragraph in get stance function Strongest among the rumours from Rajini's heydays in films is that he was romantically involved with Silk Smitha


('www.rumours.com', 'Unrelated')

In [0]:
predict_stance_for_sources('i never came yesterday',"Strongest among the rumours from Rajini's heydays in films, is that he was romantically involved with Silk Smitha.")

{'Stance': 'Unrelated'}

In [0]:
def getstance(claim,para,website):
  
  stance=predict_stance_for_sources(claim,para)
     
  return((claim,para,website,stance['Stance']))

In [0]:
os.getcwd()

'/content'

In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from keras.models import load_model



label_dict = {0:'Agree',1:'Disagree',2:'Discuss',3:'Unrelated'}
model = load_model('/content/FNC/keras_model_updated')
model._make_predict_function()
count_vectorizer = pickle.load(open('/content/FNC/count_vectorizer1.pk','rb'))
tfidf_vectorizer = pickle.load(open('/content/FNC/tfidf_vectorizer.pk','rb'))

#print(count_vectorizer)
#print(tfidf_vectorizer)
#print(model.summary())

def calculate_tfidf_score(headline,body):
    '''Calculates cosine similarity between the headline and body. '''

    headline_features = tfidf_vectorizer.transform([headline]).toarray()
    body_features = tfidf_vectorizer.transform([body]).toarray()
    cosine_similarity  = np.dot(headline_features,body_features.T)

    return cosine_similarity[0][0]

def create_input_for_model(headline,body,tfidf_score):
    '''Creates input for the model.'''

    headline_features = count_vectorizer.transform([headline]).toarray()
    body_features = count_vectorizer.transform([body]).toarray()
    input_feature = np.column_stack((headline_features,tfidf_score,body_features))

    return input_feature[0]

  
def predict_stance_for_sources(headline,body):
    '''Predict stance for events fetched from other sources similar to the given headline.'''

    tfidf_score = calculate_tfidf_score(headline=headline,body=body)
    input = create_input_for_model(headline=headline,body=body,tfidf_score=tfidf_score)

    input = input.reshape((1,-1))
    prediction = model.predict(input)
    output = np.argmax(prediction)
    stance = label_dict[output]
    response = {'Stance':stance}

    return response


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
answer=[]
websites=[]
paras=[]


for i in range(0,n_predictions):

  answer.append(format(prediction[i][0]))  #answer
  websites.append(format(prediction[i][1]))  #website
  paras.append(format(prediction[i][2]))  #paragraph


In [0]:
doc=nlp("modi is the prime minister of India. He was also the chiefminister of Gujarat")
print([doc._.coref_resolved])


['modi is the prime minister of India. modi was also the chiefminister of Gujarat']


In [0]:
paragraphs

In [0]:
# Sending a question to the pipeline and getting prediction


generatequestion("Rajini has affair with silk smitha")

Rajini ORG
question list 2 [('Rajini', 'which organization has affair with silk smitha')]


[('Rajini', 'which organization has affair with silk smitha')]

In [0]:
doc = nlp("Rajini has affair with silk smitha")
       
for ent in doc.ents:
       print(ent,ent.label_)

Rajini ORG
